<a href="https://colab.research.google.com/github/LudwigSoufi/test/blob/main/Copy_of_Refinement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function, division

import numpy as np

def get_norm_1d(tensor):
    return tensor

def get_norm_2d(tensor):
    return np.linalg.norm(tensor, axis=(0,1))

def get_dynamics(lattice, args):
    name = args[0]
    omega = float(args[1])
    print(name)
    if name == "BgkDynamics":
        return BgkDynamics(lattice, omega)
    elif name == "SmagoDynamics":
        return SmagoDynamics(lattice, omega, float(args[2]))
    else:
        raise Exception()

class BgkDynamics(object):

    def __init__(self, lattice, omega):
        self.lattice = lattice
        self.omega = omega
        self.feq = None

    def collide(self):
        return self.lattice.fin - self.omega * (self.lattice.fin - self.feq) + self.lattice.force_pop

    def get_specs(self):#for checkpointing
        return["BgkDynamics", self.omega]

class SmagoDynamics(BgkDynamics):

    def __init__(self, lattice, omega, c):
        BgkDynamics.__init__(self, lattice, omega)
        self.c = c
        self.prefactor = 0.5*(self.c*omega*3.)**0.5
        if len(lattice.fin.shape) == 3:
            self.get_norm = get_norm_2d
        elif len(lattice.fin.shape) == 2:
            self.get_norm = get_norm_1d
        else:
            raise Exception()

    def get_specs(self):#for checkpointing
        return["SmagoDynamics", self.omega, self.c]

    def collide(self):
        pi_neq = self.lattice.get_pi_neq(self.feq)
        pi_neq_norm = self.get_norm(pi_neq)
        alpha = self.prefactor / self.lattice.rho
        linear_term = alpha*pi_neq_norm
        square_term = 2.*linear_term*linear_term
        new_omega = self.omega*(1.-linear_term+square_term)
        return self.lattice.fin - new_omega * (self.lattice.fin - self.feq) + self.lattice.force_pop

class AwesomeDynamics(SmagoDynamics):

    def __init__(self, lattice, omega, c):
        BgkDynamics.__init__(self, lattice, omega)
        self.c = c

    def get_S(self):
        S = np.zeros((2,2,self.lattice.nx,self.lattice.ny))
        u = self.lattice.get_u()
        uxx = u[0,:,:]*u[0,:,:]
        uyy = u[1,:,:]*u[1,:,:]
        udiag = (u[0,:,:]*u[1,:,:]+u[1,:,:]*u[0,:,:])/2.
        S[0,0,:,:] = uxx
        S[0,1,:,:] = udiag
        S[1,0,:,:] = udiag
        S[1,1,:,:] = uyy
        return S

        #-------------------------------------------------------------------------------
# Name:        module1
# Purpose:
#
# Author:      yannt
#
# Created:     23.06.2023
# Copyright:   (c) yannt 2023
# Licence:     <your licence>
#-------------------------------------------------------------------------------

def main():
    pass

if __name__ == '__main__':
    main()
#!/usr/bin/python3
# -*- coding: utf-8 -*-
"""
In this code, for any lattice q = 9, cs2 = 1/3.
"""

import sys
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import warnings
import dynamics


def save_graph(data, name, it, colmap, bar=True, frame=True, axis=True):
    plt.clf()
    plt.imshow(np.flipud(data.transpose()), cmap=colmap, interpolation="none")
    if bar:
        try:
            plt.colorbar()
        except Warning:
            pass
    plt.savefig(name + str(it).zfill(8) + ".png")
    return name + str(it).zfill(8) + ".png"

def save_graph_only(data, name, it, bar=True, frame=True, axis=True):
    plt.clf()
    plt.imshow(np.flipud(data.transpose()), cmap=cm.Blues, interpolation="none")
    if bar:
        try:
            plt.colorbar()
        except Warning:
            pass
    plt.axis("off")
    plt.savefig(name + str(it).zfill(8) + ".png", bbox_inches='tight', pad_inches=0)
    return name + str(it).zfill(8) + ".png"

def get_feq(rho, u):
    feq = np.zeros(9)
    usqr = 1.5 * (u[0]**2 + u[1]**2) #u * u / (2cs2)
    for i in range(9):
        cu_i = 3. * (v[i,0]*u[0] + v[i,1]*u[1])
        feq[i] = rho * t[i] * (1. + cu_i + 0.5*cu_i**2 - usqr)
    return feq


###### Lattice Constants #######################################################
v = np.array([ [ 1,  1], [ 1,  0], [ 1, -1], [ 0,  1], [ 0,  0],
            [ 0, -1], [-1,  1], [-1,  0], [-1, -1] ])
v.flags.writeable = False

t = np.array([ 1./36, 1./9, 1./36, 1./9, 4./9, 1./9, 1./36, 1./9, 1./36])
t.flags.writeable = False

vv = np.zeros((9,2,2))
for a in [0,1]:
    for b in [0,1]:
        for i in range(9):
            vv[i,a,b] = v[i][a]*v[i][b]
vv.flags.writeable = False


#names from the "out" POV
names = {"topright":0, "right":1, "bottomright":2, "top":3, "rest":4,
         "bottom":5, "topleft":6, "left":7, "bottomleft":8}

opposite = np.array([8, 7, 6, 5, 4, 3, 2, 1, 0])
opposite.flags.writeable = False



###### General functions #####################################################


def get_neighbour_matrix(matrix, direction, population):
    """neighbour(i,x,y) will be the ith population of neighbour in direction (cix, ciy)."""
    i = opposite[direction]
    p = population
    return np.roll(np.roll(matrix[p,:,:], v[i,0], axis=0), v[i,1], axis=1 )

def get_neighbour_scalar(matrix, direction):
    """neighbour(x,y) will be the value of neighbour in direction (cix, ciy)."""
    i = opposite[direction]
    return np.roll(np.roll(matrix[:,:], v[i,0], axis=0), v[i,1], axis=1 )



class Lattice:

    def __init__(self, nx, ny, fx, fy, rho=None, u=None):
        self.nx = int(nx)
        self.ny = int(ny)
        self.force_pop = self.get_force_pop(fx, fy)
        self.force_pop.flags.writeable = False
        ###### Initialization ##################################################
        if u is None:
            self.u = np.zeros((2, self.nx, self.ny))
        else:
            self.u = u
        if rho is None:
            self.rho = np.ones((self.nx, self.ny))
        else:
            self.rho = rho
        self.grad_u = None
        # Initialization of the populations at equilibrium for null velocity
        self.walls = np.zeros((self.nx, self.ny), dtype=bool)
        self.fin = self.get_feq()
        self.rho = self.get_rho()
        self.u = self.get_u()
        self.dynamics = None

    def get_zero_scalar_field(self, data_type):
        return np.zeros((self.nx, self.ny), dtype=data_type)

    def write_state(self, filename):
        print("Writing state", filename)
        np.save(filename+"_fin", self.fin)
        np.save(filename+"_walls", self.walls)
        np.save(filename+"_force_pop", self.force_pop)
        specs = [self.nx, self.ny] + self.dynamics.get_specs()
        specs = [str(s) for s in specs]
        f = open(filename+"_specs.dat", "w")
        f.write(" ".join(specs))
        f.close()

    def read_state(self, filename):
        print("Reading state", filename)
        self.fin = np.load(filename+"_fin.npy")
        self.walls = np.load(filename+"_walls.npy")
        self.force_pop = np.load(filename+"_force_pop.npy")
        #dynamics
        f = open(filename+"_specs.dat", "r")
        data = f.readline()
        f.close()
        data = data.split(" ")
        nx, ny = float(data[0]), float(data[1])
        assert nx == self.nx and ny == self.ny
        self.dynamics = dynamics.get_dynamics(self, data[2:])


    def get_rho(self):
        """Returns macroscopic density rho."""
        return np.sum(self.fin, axis=0)


    def get_u(self):
        """Returns macroscopic density velocity u.
        Must be called after rho has been refreshed."""
        u = np.zeros((2, self.nx, self.ny))
        for i in range(9):
            u[0,:,:] += v[i,0] * self.fin[i,:,:]
            u[1,:,:] += v[i,1] * self.fin[i,:,:]
        u /= self.rho
        return u

    def get_feq(self):
        """Returns equilibrium distribution according to rho and u."""
        usqr = 1.5 * (self.u[0,:,:]**2 + self.u[1,:,:]**2) #u * u / (2cs2)
        feq = np.zeros((9, self.nx, self.ny)) #matrix 9*nx*ny
        for i in range(9):
            cu = 3. * (v[i,0]*self.u[0,:,:] + v[i,1]*self.u[1,:,:]) #cu/cs2, v[i,:,:] = "ith coordinate of v, for all x and for all y"
            feq[i,:,:] = self.rho*t[i] * (1 + cu + 0.5*cu**2 - usqr)
        return feq

    def get_feq_subdomain(self,x0,x1,y0,y1):
        """Returns equilibrium distribution according to rho and u."""
        usqr = 1.5 * (self.u[0,x0:x1,y0:y1]**2 + self.u[1,x0:x1,y0:y1]**2) #u * u / (2cs2)
        feq = np.zeros((9, x1-x0, y1-y0)) #matrix 9*nx*ny
        for i in range(9):
            cu = 3. * (v[i,0]*self.u[0,x0:x1,y0:y1] + v[i,1]*self.u[1,x0:x1,y0:y1]) #cu/cs2, v[i,:,:] = "ith coordinate of v, for all x and for all y"
            feq[i,:,:] = self.rho[x0:x1,y0:y1]*t[i] * (1 + cu + 0.5*cu**2 - usqr)
        return feq

    def get_feq_xy(self, x, y):
        feq = np.zeros(9)
        usqr = 1.5 * (self.u[0,x,y]**2 + self.u[1,x,y]**2) #u * u / (2cs2)
        for i in range(9):
            cu_i = 3. * (v[i,0] * self.u[0,x,y] + v[i,1] * self.u[1,x,y])
            feq[i] = RHO_ATM * t[i] * (1. + cu_i + 0.5*cu_i**2 - usqr)
        return feq

##    def get_pi_neq(self, feq):
##        pi = np.zeros((2,2,self.nx,self.ny))
##        for a in [0,1]: #can be optimized: hardcode 00 01 10 11
##            for b in [0,1]:
##                for i in range(9):
##                    pi[a,b,:,:] += v[i][a]*v[i][b]*(self.fin[i,:,:] - feq[i,:,:])
##        return pi

##    def get_pi_neq(self, feq):
##        pi = np.zeros((2,2,self.nx,self.ny))
##        for i in range(9):
##            fneq = self.fin[i,:,:] - feq[i,:,:]
##            pi[0,0,:,:] += v[i][0]*v[i][0]*fneq #+= v[i,0,0]*fneq
##            pi[0,1,:,:] += v[i][0]*v[i][1]*fneq
##            pi[1,0,:,:] += v[i][1]*v[i][0]*fneq
##            pi[1,1,:,:] += v[i][1]*v[i][1]*fneq
##        return pi

    def get_pi_neq(self, feq):
        pi = np.zeros((2,2,self.nx,self.ny))
        for i in range(9):
            fneq = self.fin[i,:,:] - feq[i,:,:]
            pi[0,0,:,:] += vv[i,0,0]*fneq #+= v[i,0,0]*fneq
            pi[0,1,:,:] += vv[i,0,1]*fneq
            pi[1,0,:,:] += vv[i,1,0]*fneq
            pi[1,1,:,:] += vv[i,1,1]*fneq
        return pi

    def get_force_pop(self, fx, fy):
        force = np.zeros((9, self.nx, self.ny))
        for i in range(9):
            force[i, :, :] = 3. * t[i] * (v[i, 0]*fx + v[i, 1]*fy) #t * (v*F) / cs2
        return force

    def adjacent(self, x, y):
        for i in range(9):
            next_x = (x+v[i, 0]) % self.nx
            next_y = (y+v[i, 1]) % self.ny
            yield next_x, next_y



    def get_grad_u(self):
        ux = self.u[0,:,:]/2
        uy = self.u[1,:,:]/2
        duxdx = np.roll(ux, 1, axis=0) - np.roll(ux, -1, axis=0)
        duydy = np.roll(uy, 1, axis=1) - np.roll(uy, -1, axis=1)
        duxdy = np.roll(ux, 1, axis=1) - np.roll(ux, -1, axis=1)
        duydx = np.roll(uy, 1, axis=0) - np.roll(uy, -1, axis=0)
        return duxdx, duydy, duxdy, duydx

    def zou_he_right_missing(self, x, u):
        hi = 0
        hf = self.ny-1
        for y in range(hi,hf):
            self.u[0,x,y] = u[0]
            self.u[1,x,y] = u[1]
        col_l = np.array([0, 1, 2]) #(left)
        col_m = np.array([3, 4, 5]) #(middle)
        col_r = np.array([6, 7, 8]) #(right)
        sum_m = np.sum(self.fin[col_m,x,hi:hf], axis=0)
        sum_l = np.sum(self.fin[col_l,x,hi:hf], axis=0)
        self.rho[x,hi:hf] = (sum_m + 2*sum_l)/(1.-self.u[0,x,hi:hf])
        #Compute equilibrium.
        feq = self.get_feq()
        fneq = self.fin[[2,1,0],x,hi:hf] - feq[[2,1,0],x,hi:hf] #opp. to left
        self.fin[col_r,x,hi:hf] = feq[col_r,x,hi:hf] + fneq
        feq.flags.writeable = False
        return feq

    def zou_he_left_missing(self, x, u):
        hi = HI_LEFT
        hf = HF_LEFT
        for y in range(hi,hf):
            self.u[0,x,y] = u[0]
            self.u[1,x,y] = u[1]
        col_l = np.array([0, 1, 2]) #(left)
        col_m = np.array([3, 4, 5]) #(middle)
        col_r = np.array([6, 7, 8]) #(right)
        sum_m = np.sum(self.fin[col_m,x,hi:hf], axis=0)
        sum_r = np.sum(self.fin[col_r,x,hi:hf], axis=0)
        self.rho[x,hi:hf] = (sum_m + 2*sum_r)/(1.-self.u[0,x,hi:hf])
        #Compute equilibrium.
        feq = self.get_feq()
        fneq = self.fin[[8,7,6],x,hi:hf] - feq[[8,7,6],x,hi:hf] #opp. to left
        self.fin[col_l,x,hi:hf] = feq[col_l,x,hi:hf] + fneq
        feq.flags.writeable = False
        return feq

    def iterate(self, conditions=()):
        #Compute macroscopic variables: density and velocity.
        self.rho = self.get_rho()
        self.u = self.get_u()
        # # zou he boundary conditions:
        # for side, x, u in conditions:
        #     if side == "left":
        #         feq = self.zou_he_left_missing(x, u)
        #     elif side == "right":
        #         feq = self.zou_he_right_missing(x, u)
        #     else:
        #         raise ValueError("Implementend bc sides are left and right.")
        # if not conditions:
        #     feq = self.get_feq()
        #     feq.flags.writeable = False
        # else:
        #     assert False
        #####################################################################
        #Equilibrium boundary conditions
        # self.u[0,0,:] = 0. #left
        # self.u[1,0,:] = 0 #left
        # self.u[0,self.nx-1,:] = 0. #right
        # self.u[1,self.nx-1,:] = 0. #right
        # self.u[0,:,0] = 0. #top
        # self.u[1,:,0] = 0. #top
        # self.u[0,:,self.ny-1] = 0. #bottom
        # self.u[1,:,self.ny-1] = 0. #bottom
        # feq = self.get_feq()
        # feq.flags.writeable = False
        # self.fin[:,:,self.ny-1] = feq[:,:,self.ny-1]
        # ... and so on
        #######################################################################
        feq = self.get_feq()
        feq.flags.writeable = False
        self.dynamics.feq = feq
        #Collision step.
        fout = self.dynamics.collide()
        #Half-way bounce-back condition for obstacle.
        # for i in range(9):
        #     fout[i, self.walls] = self.fin[opposite[i], self.walls]
        #Moving wall:
        for wall_sites, u_wall in self.moving_walls:
            for i in range(9):
                I = opposite[i]
                f =  self.fin[I, wall_sites]
                ci_dot_uwall = v[I][0]*u_wall[0] + v[I][1]*u_wall[1]
                #formula from kruger: 2*t[I]*rho_w*ci_dot_uwall/cs2
                #here 1/cs2 = 3 and rho_wall = 1 (incompr flow)
                f -= 6*t[I]*ci_dot_uwall
                fout[i, wall_sites] = f
        #streaming
        for i in range(9):
            self.fin[i,:,:] = get_neighbour_matrix(fout, opposite[i], i)
        #Equilibrium boundary conditions
        # self.u[0,0,:] = 0. #left
        # self.u[1,0,:] = 0 #left
        # #
        self.rho[NO_MANS_LAND,:] = 1.
        self.u[0,NO_MANS_LAND,:] = 0. #right
        self.u[1,NO_MANS_LAND,:] = 0. #right
        #
        # self.u[0,:,0] = 0. #top
        # self.u[1,:,0] = 0. #top
        #
        self.rho[:,NO_MANS_LAND] = 1.
        self.u[0,:,NO_MANS_LAND] = 0. #bottom
        self.u[1,:,NO_MANS_LAND] = 0. #bottom
        #

        # self.fin[:,:,0] = feq[:,:,0] #top
        feq = self.get_feq_subdomain(NO_MANS_LAND,NO_MANS_LAND+1,0,self.ny)
        self.fin[:,NO_MANS_LAND:NO_MANS_LAND+1,:] = feq #right
        # self.fin[:,0,:] = feq[:,0,:] #left
        feq = self.get_feq_subdomain(0,self.nx,NO_MANS_LAND,NO_MANS_LAND+1)
        self.fin[:,:,NO_MANS_LAND:NO_MANS_LAND+1] = feq #bottom
        return fout


def normalize(a):
    m = np.min(a)
    M = np.max(a)
    if M-m == 0:
        return a
    return (a-m) / (M-m)



force_y = 0.
force_x = 0.

N = 150
Re = 5000.
##N = int(float(sys.argv[1]))
##Re = float(sys.argv[2])
uwall = 20e-2
nx = N
ny = N

m = 1
k_avg = np.zeros((N,N))
vorticity_avg = np.zeros((N,N))
fakh_avg = np.zeros((N,N))
Q_avg = np.zeros((N,N))
N_avg = 0

#We want a convective scaling : dx/dt = constant
#Re = uL/nu <==> nu = uL/Re
#But nu = cs2 (tau - 0.5) <==> tau = uL/(Re*cs2) + 0.5
cs2 = 1./3.
tau = uwall * nx / (Re * cs2) + 0.5
##tau = 1.
Re = cs2 * (tau - 0.5) / (uwall*nx)
omega = 1./tau

print("Re =", Re)
print("N =", N)
print("uwall =", uwall)
print("tau =", tau)

lat = Lattice(nx, ny, force_x, force_y)
# lat.dynamics = dynamics.BgkDynamics(lat, omega)
lat.dynamics = dynamics.SmagoDynamics(lat, omega, 0.14)

# Zou-He walls
# HI_LEFT = 1
# HF_LEFT = lat.ny-2
# cond_left = ("left", 0, (0,uwall))
# boundary_conditions = [cond_left]
boundary_conditions = None

# bounceback walls
# lat.walls[0,:] = True #left
# lat.walls[lat.nx-1,:] = True #right
# lat.walls[:,0] = True #top
# lat.walls[:,lat.ny-1] = True #bottom

# lat.walls[lat.nx-2,:] = True
# lat.walls[:,1] = True
# lat.walls[:,lat.ny-2] = True

NO_MANS_LAND = N-1
#moving walls: CAUTION : never define 2 conditions on any cell
lat.moving_walls = []

bottom_wall = lat.get_zero_scalar_field(bool)
bottom_wall[:,NO_MANS_LAND-1] = True
bottom_wall_condition = (bottom_wall, (uwall, 0.))
lat.moving_walls += [bottom_wall_condition]

top_wall = lat.get_zero_scalar_field(bool)
top_wall[:,0] = True
top_wall_condition = (top_wall, (0., 0.))
lat.moving_walls += [top_wall_condition]

left_wall = lat.get_zero_scalar_field(bool)
left_wall[0,1:NO_MANS_LAND-2] = True
left_wall_condition = (left_wall, (0., 0.))
lat.moving_walls += [left_wall_condition]

right_wall = lat.get_zero_scalar_field(bool)
right_wall[NO_MANS_LAND-1,1:NO_MANS_LAND-2] = True
right_wall_condition = (right_wall, (0., 0.))
lat.moving_walls += [right_wall_condition]


for it in range(100000):
    lat.iterate(boundary_conditions)
    u_sqrt = np.sqrt(lat.u[0]**2 + lat.u[1]**2)[0:NO_MANS_LAND-1,0:NO_MANS_LAND-1]
    if it%100 == 0:
        print(it)
        save_graph(u_sqrt, "avel", it, cm.Blues)



ModuleNotFoundError: No module named 'dynamics'

ModuleNotFoundError: No module named 'dynamics'